In [212]:
import os 
import logging 
import gzip 
import numpy as np 
import pandas as pd

In [213]:
def ungzip(file_path):
    """
    Take a file path and ungzip it
    """
    # TODO: there should probably be some error checks here
    ungzipped_file_path = file_path.rstrip('.gz')
    with gzip.open(file_path, 'rb') as gz_file:
        file_content = gz_file.read()
    return file_content.decode('utf-8')


#dns log 
{
  "ts": 1591367999.306059,
  "uid": "CMdzit1AMNsmfAIiQc",
  "id.orig_h": "192.168.4.76",
  "id.orig_p": 36844,
  "id.resp_h": "192.168.4.1",
  "id.resp_p": 53,
  "proto": "udp",
  "trans_id": 8555,
  "query": "testmyids.com",
  "qclass": 1,
  "qclass_name": "C_INTERNET",
  "qtype": 28,
  "qtype_name": "AAAA",
  "rcode": 0,
  "rcode_name": "NOERROR",
  "AA": false,
  "TC": false,
  "RD": true,
  "RA": false,
  "Z": 0,
  "rejected": false
}
https://docs.zeek.org/en/master/logs/dns.html 

## conn.log

In [17]:
features = ['id.orig_p', "id.resp_p", "proto", "conn_state", "missed_bytes",
                "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]

In [18]:
data_list = []
for line in json_data_file.splitlines():
    # log_entry is now a single json log from the file
    log_entry = json.loads(line.strip())
    data_list.append([log_entry[feature] for feature in features])
np_arr = np.array(data_list)

In [25]:
df = pd.DataFrame(data_list, columns=features)

In [26]:
df

id.orig_p  id.resp_p proto conn_state  missed_bytes  orig_pkts  \
0       65363         53   udp         SF             0          1   
1       51657         53   udp         SF             0          1   
2       51400        443   tcp       RSTR             0          2   
3        5353       5353   udp         S0             0          2   
4        5353       5353   udp         S0             0          2   
..        ...        ...   ...        ...           ...        ...   
60      51473        443   tcp         SF             0         12   
61      51480        443   tcp         SF             0         12   
62      51481        443   tcp         SF             0         12   
63      51482        443   tcp         SF             0         12   
64      51445        443   tcp      RSTRH             0          1   

    orig_ip_bytes  resp_pkts  resp_ip_bytes  
0              62          1            169  
1              62          1            142  
2             128          2             80  
3             469          0              0  
4             509          0              0  
..            ...        ...            ...  
60           1921          9           5408  
61           1921          9           5408  
62           1921          9           5408  
63           2484          9           5408  
64             40          1             40  

[65 rows x 9 columns]

In [22]:
log_entry.keys()

dict_keys(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes'])

In [54]:
#preprocess functions 
def is_private_ip(ip_str):
    """
    Takes an IP string and returns whether the IP is private or not per RFC 1918.

    Parameters
    ----------
    ip_str: str
        String of an IP address.

    Returns
    -------
    bool: a bool of whether or not the IP is private. 
    """
    octets = [int(x) for x in ip_str.split(".")]
    if octets[0] == 10 \
            or (octets[0] == 172 and 16 <= octets[1] <= 31) \
            or (octets[0] == 192 and octets[1] == 168):
        return True
    else:
        return False

def get_traffic_direction(source_ip, destination_ip):
    """
    Takes a source and destination IP address and returns the direction of the traffic.
    Please ensure the source and destination are correct as this is useless without the verification of the parameters.

    Parameters
    ----------
    source_ip: str
        Source IP address of the flow.
    destination_ip: str
        Destination IP address of the flow.
    
    Returns
    -------
    str: string indicating the direction. Can be 'internal', 'outgoing', 'incoming' or 'external'.
    """
    if is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "internal"
    elif is_private_ip(source_ip) and not is_private_ip(destination_ip):
        return "outgoing"
    elif not is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "incoming"
    else:
        return "external"
    
def create_history_variable(new_df):
    # break out history variable
    
    #fill NaNs with 'N'
    # new_df['history'] = new_df['history'].fillna('N') 
    if 'history' not in new_df.columns:
            new_df['history'] = 'N'

    new_df['history_has_S'] = new_df['history'].apply(lambda x: 1 if "S" in x else 0)
    new_df['history_has_h'] = new_df['history'].apply(lambda x: 1 if "h" in x else 0)
    new_df['history_has_A'] = new_df['history'].apply(lambda x: 1 if "A" in x else 0)
    new_df['history_has_D'] = new_df['history'].apply(lambda x: 1 if "D" in x else 0)
    new_df['history_has_a'] = new_df['history'].apply(lambda x: 1 if "a" in x else 0)
    new_df['history_has_d'] = new_df['history'].apply(lambda x: 1 if "d" in x else 0)
    new_df['history_has_F'] = new_df['history'].apply(lambda x: 1 if "F" in x else 0)
    new_df['history_has_f'] = new_df['history'].apply(lambda x: 1 if "f" in x else 0)
    new_df['history_has_N'] = new_df['history'].apply(lambda x: 1 if "N" in x else 0)
    new_df = new_df.drop(columns='history')

    # if 'history'in new_df.columns:
    #     new_df = new_df[new_df['history'].notna()].copy() #filters out rows where the 'history' column is null
    #     new_df['history_has_S'] = new_df['history'].apply(lambda x: 1 if "S" in x else 0)
    #     new_df['history_has_h'] = new_df['history'].apply(lambda x: 1 if "h" in x else 0)
    #     new_df['history_has_A'] = new_df['history'].apply(lambda x: 1 if "A" in x else 0)
    #     new_df['history_has_D'] = new_df['history'].apply(lambda x: 1 if "D" in x else 0)
    #     new_df['history_has_a'] = new_df['history'].apply(lambda x: 1 if "a" in x else 0)
    #     new_df['history_has_d'] = new_df['history'].apply(lambda x: 1 if "d" in x else 0)
    #     new_df['history_has_F'] = new_df['history'].apply(lambda x: 1 if "F" in x else 0)
    #     new_df['history_has_f'] = new_df['history'].apply(lambda x: 1 if "f" in x else 0)
    #     new_df = new_df.drop(columns='history')
    # else: 
    #     new_df['history_has_S'] = 0
    #     new_df['history_has_h'] = 0
    #     new_df['history_has_A'] = 0
    #     new_df['history_has_D'] = 0
    #     new_df['history_has_a'] = 0
    #     new_df['history_has_d'] = 0
    #     new_df['history_has_F'] = 0
    #     new_df['history_has_f'] = 0
    # drop ipv6 data
    if 'id.orig_h'in new_df.columns:
        new_df = new_df[new_df['id.orig_h'].str.contains("::") == False]
    return new_df 

def create_broadcast_variable(new_df):
    # create broadcast variable
    #255 is the broadcast address for ipv4(#TODO : ask Diego)
    if 'id.resp_h' in new_df.columns:
        new_df['is_destination_broadcast'] = new_df['id.resp_h'].apply(lambda x: 1 if "255" in x[-3:] else 0) 
    return new_df

def create_direction_variable(new_df):
    #create traffic direction variable
    if 'traffic_direction' in new_df.columns:
        new_df['traffic_direction']        = new_df.apply(lambda x: get_traffic_direction(x['id.orig_h'], x['id.resp_h']), axis=1) 
    return new_df

def one_hot_encode(df, column_name):
    new_df = pd.get_dummies(data=df, columns=column_name)
    return new_df

def duration_to_numerical(new_df):
    # Convert duration to string
    new_df['duration'] = new_df['duration'].astype(str)
    # Extract the time portion (HH:MM:SS.mmmmmm) from the 'duration' column
    new_df['duration'] = new_df['duration'].str.extract(r'\d days (.*)')
    # Convert the time portion to a numerical format (float)
    new_df['duration'] = pd.to_timedelta(new_df['duration']).dt.total_seconds()
    return new_df 

def drop_columns(new_df, columns_to_drop):
    columns_to_drop_existing = [col for col in columns_to_drop if col in new_df.columns]
    new_df.drop(columns=columns_to_drop_existing, axis=1, inplace=True)
    return new_df


#TODO: create a function that takes in a dataframe and perform the preprocessing steps on it
def preprocess(new_df):
    
    # Drop unnecessary columns 
    columns_to_drop = ['ts','uid','local_orig', 'local_resp']
    new_df.drop(columns_to_drop, axis=1, inplace=True)

    # create history, broadcast, traffic_direction variables
    new_df = create_history_variable(new_df)
    new_df = create_broadcast_variable(new_df)
    new_df = create_direction_variable(new_df)

    # one hot encode categorical variables
    #TODO : discuss with Diego, if there's a better way to do this. since, input dataset may have different conn state, that means the columns would be different. 
    column_name = ['conn_state', "proto", "traffic_direction" , "service"]
    for col in column_name:
        if col in new_df.columns:
            new_df = one_hot_encode(new_df, [col])
    new_df = new_df.drop(columns=['id.orig_h', 'id.resp_h'])

    return new_df


def fill_na(new_df):
    
    #Fill Nans with 0s : duration, orig_bytes resp_bytes
    # Specify the columns you want to fill with zeros
    columns_to_fill_with_zeros = ['duration', 'orig_bytes', 'resp_bytes']
    # Check if columns exist; if not, create and fill with zeros
    for col in columns_to_fill_with_zeros:
        if col not in new_df.columns:
            new_df[col] = 0
    new_df[columns_to_fill_with_zeros] = new_df[columns_to_fill_with_zeros].fillna(0)
    

    #Fill Nans with 'Other' : service
    columns_to_fill_with_other = ['service']
    if 'service' in new_df.columns:
    # new_df['service'].fillna('other', inplace=True)
        new_df['service'] = new_df['service'].fillna('other')
        
    return new_df

def makedf_samecol(new_df):
    #Create these columns if they are not present in the original df and fill them with 0s. 
    # Ensure that all the specified columns are present even if they are not present in the original df. 
    cols = ['conn_state_OTH', 'conn_state_REJ','conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR','conn_state_RSTRH', 
        'conn_state_S0', 'conn_state_S1', 'conn_state_S2','conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR',
        'proto_tcp', 'proto_udp',
        'service_dhcp', 'service_dns','service_http', 'service_irc','service_ntp',
        'service_other', 'service_ssh','service_ssl',
        'traffic_direction_external','traffic_direction_incoming', 
        'traffic_direction_internal','traffic_direction_outgoing']
    for col in cols:
        if col not in new_df.columns:
            new_df[col] = 0
    return new_df


In [59]:
def preprocess_json(json_batch):
    """
    This function receives a json batch from the main control flow of the train 
    functions. It should convert the json_batch to a numpy 2D array, apply necessary transformations,
    then return it. 

    Note: the input is only one unzipped json file. 
    """
    # TODO: add the featureset here 
    # TODO: should we move this feature set somewhere else?
    features = ['id.orig_p', "id.resp_p", "proto", "conn_state", "missed_bytes",
                "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]
    # add the following features ['duration', 'history']
    # TODO: @olive please run the script as is, it should work.
    # However, some log records in json do not have duration or history fields.
    # Please catch this error, and if there is no duration, add a duration of 0 to the record. 
    # If there is no history, add a history, with the value "N"
    data_list = []
    for line in json_batch.splitlines():
        # log_entry is now a single json log from the file
        log_entry = json.loads(line.strip())
        data_list.append([log_entry[feature] for feature in features])
    # np_arr = np.array(data_list)
    
    # TODO: apply transformations based on last semesters work
    #Re-use the preprocess function from last sem by Zoe. 
    #TODO: optimize the code via removing pandas
    new_df = pd.DataFrame(data_list, columns=features) 
    #Fill NaNs with 0s : duration, orig_bytes resp_bytes, if there are no columns, create one and fill with 0s 
    new_df = fill_na(new_df) 
    # Drop unnecessary columns 
    new_df = drop_columns(new_df, ['ts','uid','local_orig', 'local_resp'])
    
    # create history, broadcast, traffic_direction variables
    new_df = create_history_variable(new_df)
    new_df = create_broadcast_variable(new_df)
    new_df = create_direction_variable(new_df)

    # one hot encode categorical variables
    column_name = ['conn_state', "proto", "traffic_direction" , "service"]
    for col in column_name:
        if col in new_df.columns:
            new_df = one_hot_encode(new_df, [col])
    # new_df = new_df.drop(columns=['id.orig_h', 'id.resp_h'])

    new_df = drop_columns(new_df, ['id.orig_h', 'id.resp_h'])

    # make sure the columns are the same as the original df
    new_df = makedf_samecol(new_df)
    # new_df = new_df.drop(columns=['orig_l2_addr','resp_l2_addr'])
    new_df = drop_columns(new_df, ['orig_l2_addr','resp_l2_addr'])

    # Convert DataFrame to NumPy array
    np_arr = new_df.to_numpy()# np_arr is now a numpy 2D array
    
    logging.info("Hello from preprocess_json. Please implement me :)")
    return np_arr

In [60]:
current_dir_path = '/usr/local/logs/2024-02-12'
if not os.path.islink(current_dir_path):
    # sub_dir is now any given historical data directory 
    logging.info(f"Checking {current_dir_path}")
    for file in os.listdir(current_dir_path):
        # file is now any given file in the historical data directory
        current_file_path = os.path.join(current_dir_path, file)
        if "conn." in file:
            # get the whole file in memory
            logging.info(f"Opening file {current_file_path}")
            json_data_file = ungzip(current_file_path)
            print(current_file_path)
            np_arr = preprocess_json(json_data_file)
            

            # # process json and give back a np_array (in utils)
            # np_arr = preprocess_json(json_data_file)
            # train_batch(kit, np_arr)

/usr/local/logs/2024-02-12/conn.07:09:20-08:00:00.log.gz
Index(['id.orig_p', 'id.resp_p', 'proto', 'conn_state', 'missed_bytes',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes'],
      dtype='object')
[[63393  5228     0 ...     0     0     0]
 [64457    53     0 ...     0     0     0]
 [53988    53     0 ...     0     0     0]
 ...
 [56459    53     0 ...     0     0     0]
 [54084    53     0 ...     0     0     0]
 [60681    53     0 ...     0     0     0]]
/usr/local/logs/2024-02-12/conn.05:13:22-06:02:53.log.gz
Index(['id.orig_p', 'id.resp_p', 'proto', 'conn_state', 'missed_bytes',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes'],
      dtype='object')
[[59403   137     0 ...     0     0     0]
 [35489   137     0 ...     0     0     0]
 [44385  5353     0 ...     0     0     0]
 ...
 [57294  5353     0 ...     0     0     0]
 [42608  5353     0 ...     0     0     0]
 [63333   443     0 ...     0     0     0]]
/usr/local/logs/2024-02-12/conn.09

## dns

In [294]:
current_dir_path = '/usr/local/logs/2024-02-12'
if not os.path.islink(current_dir_path):
    # sub_dir is now any given historical data directory 
    logging.info(f"Checking {current_dir_path}")
    for file in os.listdir(current_dir_path):
        # file is now any given file in the historical data directory
        current_file_path = os.path.join(current_dir_path, file)
        if "dns." in file: #conn.
            # get the whole file in memory
            logging.info(f"Opening file {current_file_path}")
            json_data_file = ungzip(current_file_path)
            # print(current_file_path)
            # print(json_data_file)

In [295]:
import json 
for line in json_data_file.splitlines():
        # log_entry is now a single json log from the file
        log_entry = json.loads(line.strip())
        print(log_entry)

{'ts': 1707768003.869346, 'uid': 'CS9fzl4EIr1i9ibne5', 'id.orig_h': '10.19.235.169', 'id.orig_p': 65501, 'id.resp_h': '128.122.0.71', 'id.resp_p': 53, 'proto': 'udp', 'trans_id': 3613, 'rtt': 0.01270914077758789, 'query': 'guzzoni.apple.com', 'qclass': 1, 'qclass_name': 'C_INTERNET', 'qtype': 65, 'qtype_name': 'HTTPS', 'rcode': 0, 'rcode_name': 'NOERROR', 'AA': False, 'TC': False, 'RD': True, 'RA': True, 'Z': 0, 'answers': ['guzzoni-apple-com.v.aaplimg.com'], 'TTLs': [3321.0], 'rejected': False}
{'ts': 1707768003.869397, 'uid': 'C7KsDehrLNsfmgqo', 'id.orig_h': '10.19.235.169', 'id.orig_p': 53141, 'id.resp_h': '128.122.0.71', 'id.resp_p': 53, 'proto': 'udp', 'trans_id': 5769, 'rtt': 0.012659072875976562, 'query': 'guzzoni.apple.com', 'qclass': 1, 'qclass_name': 'C_INTERNET', 'qtype': 1, 'qtype_name': 'A', 'rcode': 0, 'rcode_name': 'NOERROR', 'AA': False, 'TC': False, 'RD': True, 'RA': True, 'Z': 0, 'answers': ['guzzoni-apple-com.v.aaplimg.com', '34.225.66.6'], 'TTLs': [3321.0, 100.0], '

In [296]:
#utils 
import ipaddress
def one_hot_encode(df, column_name):
    for col in column_name:
        if col in df.columns:
            df = pd.get_dummies(data=df, columns=[col])
    return df

def create_broadcast_variable(new_df):
    # create broadcast variable
    # can have more than one broadcast address
    #255 is the broadcast address for ipv4 
    if 'id.resp_h' in new_df.columns:
        new_df['is_destination_broadcast'] = new_df['id.resp_h'].apply(lambda x: 1 if "255" in x[-3:] else 0) 
    return new_df

def create_direction_variable(new_df):
    # #create traffic direction variable
    new_df['traffic_direction']        = new_df.apply(lambda x: get_traffic_direction(x['id.orig_h'], x['id.resp_h']), axis=1) 
    return new_df


def get_traffic_direction(source_ip, destination_ip):
    """
    Takes a source and destination IP address and returns the direction of the traffic.
    Please ensure the source and destination are correct as this is useless without the verification of the parameters.

    Parameters
    ----------
    source_ip: str
        Source IP address of the flow.
    destination_ip: str
        Destination IP address of the flow.
    
    Returns
    -------
    str: string indicating the direction. Can be 'internal', 'outgoing', 'incoming' or 'external'.
    """
    src_ip = ipaddress.ip_address(source_ip) 
    dest_ip = ipaddress.ip_address(destination_ip) 
    if src_ip.version == 6 or dest_ip.version ==6:
        return "IPv6"
    
    if is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "internal"
    elif is_private_ip(source_ip) and not is_private_ip(destination_ip):
        return "outgoing"
    elif not is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "incoming"
    else:
        return "external"

# def is_private_ip(ip_str):
#     """
#     Takes an IP string and returns whether the IP is private or not per RFC 1918.

#     Parameters
#     ----------
#     ip_str: str
#         String of an IP address.

#     Returns
#     -------
#     bool: a bool of whether or not the IP is private. 
#     """
#     octets = [int(x) for x in ip_str.split(".")]
#     if octets[0] == 10 \
#             or (octets[0] == 172 and 16 <= octets[1] <= 31) \
#             or (octets[0] == 192 and octets[1] == 168):
#         return True
#     else:
#         return False
    


def is_private_ip(ip_str):
    """
    Takes an IP string and returns whether the IP is private or not per RFC 1918.

    Parameters
    ----------
    ip_str: str
        String of an IP address.

    Returns
    -------
    bool: a bool of whether or not the IP is private.
    """
    try:
        ip = ipaddress.ip_address(ip_str)
        if ip.version == 4:
            return ip.is_private
        else:
            return False  # Ignore IPv6 addresses
    except ValueError:
        return False  # Invalid IP address format

def makedf_samecol(cols, new_df):
    #Create these columns if they are not present in the original df and fill them with 0s. 
    # Ensure that all the specified columns are present even if they are not present in the original df. 

    for col in cols:
        if col not in new_df.columns:
            new_df[col] = 0
    return new_df[cols]

#conn + Four other logs: dns, http, ssl, ssh 
#for dns: proto (str), rtt(float), qclass(num as type), qtype(str), rcode(numeric 0-9), AA (str, true or false), TC(str, true or false), RD(str, true or false), RA(str, true or false), Z, rejected(str, true or false) 
TODO: check what is the value of Z, and do we need domain name. check what might not be in the logs (so far: rcode,rtt )
#for http: trans_depth, method, 


In [297]:
features = ['id.orig_h', "id.resp_h", "proto", "rtt","qclass_name", "qtype_name","rcode_name",
                "AA","TC","RD","RA", "rejected"]
        
data_list = []
for line in json_data_file.splitlines():
    # log_entry is now a single json log from the file
    log_entry = json.loads(line.strip())
    
    # Check if each feature is present in the log_entry
    feature_values = [log_entry.get(feature, None) for feature in features]
    data_list.append(feature_values)

df = pd.DataFrame(data_list, columns=features)

In [298]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'proto', 'rtt', 'qclass_name', 'qtype_name',
       'rcode_name', 'AA', 'TC', 'RD', 'RA', 'rejected'],
      dtype='object')

In [299]:
df['rcode_name'].value_counts()
#there are total of 1984 rows 
# NOERROR     1860
# NXDOMAIN      12

NOERROR     1860
NXDOMAIN      12
Name: rcode_name, dtype: int64

In [300]:
#This code checks for null values in each feature
has_null = []
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")    
    if null_count:
        has_null.append(feature)

# Create a variable for each feature that contains null, with the column name "has_null_featurename"
for feature in has_null: 
    df[f'has_{feature}'] = df[feature].notnull().astype(int)


Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for proto: 0
Null count for rtt: 736
Null count for qclass_name: 73
Null count for qtype_name: 73
Null count for rcode_name: 112
Null count for AA: 0
Null count for TC: 0
Null count for RD: 0
Null count for RA: 0
Null count for rejected: 0


In [301]:
has_null 

['rtt', 'qclass_name', 'qtype_name', 'rcode_name']

In [302]:
#create broadcast and direction variables
df = create_broadcast_variable(df)
df = create_direction_variable(df)

In [303]:
print(df['traffic_direction'].shape)
print(df['traffic_direction'].value_counts())
print(df['traffic_direction'].isnull().sum())

(1984,)
outgoing    1682
IPv6         302
Name: traffic_direction, dtype: int64
0


In [305]:
#one hot encode qtype, qclass, rcode_name
column_name = ['proto','qtype_name','qclass_name','rcode_name','traffic_direction']
df = one_hot_encode(df, column_name)

In [306]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'rtt', 'AA', 'TC', 'RD', 'RA', 'rejected',
       'has_rtt', 'has_qclass_name', 'has_qtype_name', 'has_rcode_name',
       'is_destination_broadcast', 'proto_udp', 'qtype_name_*', 'qtype_name_A',
       'qtype_name_AAAA', 'qtype_name_HTTPS', 'qtype_name_PTR',
       'qclass_name_C_INTERNET', 'qclass_name_qclass-32769',
       'rcode_name_NOERROR', 'rcode_name_NXDOMAIN', 'traffic_direction_IPv6',
       'traffic_direction_outgoing'],
      dtype='object')

In [307]:
#encode boolean features 
boolean_to_convert = ['AA', 'TC', 'RD', 'RA', 'rejected']
df[boolean_to_convert] = df[boolean_to_convert].astype(int)


In [308]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'rtt', 'AA', 'TC', 'RD', 'RA', 'rejected',
       'has_rtt', 'has_qclass_name', 'has_qtype_name', 'has_rcode_name',
       'is_destination_broadcast', 'proto_udp', 'qtype_name_*', 'qtype_name_A',
       'qtype_name_AAAA', 'qtype_name_HTTPS', 'qtype_name_PTR',
       'qclass_name_C_INTERNET', 'qclass_name_qclass-32769',
       'rcode_name_NOERROR', 'rcode_name_NXDOMAIN', 'traffic_direction_IPv6',
       'traffic_direction_outgoing'],
      dtype='object')

In [309]:
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")   

Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for rtt: 736
Null count for AA: 0
Null count for TC: 0
Null count for RD: 0
Null count for RA: 0
Null count for rejected: 0
Null count for has_rtt: 0
Null count for has_qclass_name: 0
Null count for has_qtype_name: 0
Null count for has_rcode_name: 0
Null count for is_destination_broadcast: 0
Null count for proto_udp: 0
Null count for qtype_name_*: 0
Null count for qtype_name_A: 0
Null count for qtype_name_AAAA: 0
Null count for qtype_name_HTTPS: 0
Null count for qtype_name_PTR: 0
Null count for qclass_name_C_INTERNET: 0
Null count for qclass_name_qclass-32769: 0
Null count for rcode_name_NOERROR: 0
Null count for rcode_name_NXDOMAIN: 0
Null count for traffic_direction_IPv6: 0
Null count for traffic_direction_outgoing: 0


In [310]:
#fillna with 0s:rtt
columns_to_fill_with_zeros = ['rtt']
df[columns_to_fill_with_zeros] = df[columns_to_fill_with_zeros].fillna(0)

In [311]:
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")   

Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for rtt: 0
Null count for AA: 0
Null count for TC: 0
Null count for RD: 0
Null count for RA: 0
Null count for rejected: 0
Null count for has_rtt: 0
Null count for has_qclass_name: 0
Null count for has_qtype_name: 0
Null count for has_rcode_name: 0
Null count for is_destination_broadcast: 0
Null count for proto_udp: 0
Null count for qtype_name_*: 0
Null count for qtype_name_A: 0
Null count for qtype_name_AAAA: 0
Null count for qtype_name_HTTPS: 0
Null count for qtype_name_PTR: 0
Null count for qclass_name_C_INTERNET: 0
Null count for qclass_name_qclass-32769: 0
Null count for rcode_name_NOERROR: 0
Null count for rcode_name_NXDOMAIN: 0
Null count for traffic_direction_IPv6: 0
Null count for traffic_direction_outgoing: 0


In [ ]:
#TODO: to be confirmed once EDA is done
dns_cols = ['rtt', 'AA', 'TC', 'RD', 'RA', 'rejected',
       'has_rtt', 'has_qclass_name', 'has_qtype_name', 'has_rcode_name',
       'is_destination_broadcast', 
       'proto_tcp', 'proto_udp',
       'qtype_name_*', 'qtype_name_A',
       'qtype_name_AAAA', 'qtype_name_HTTPS', 'qtype_name_PTR',
       'qclass_name_C_INTERNET', 'qclass_name_qclass-32769',
       'rcode_name_NOERROR', 'rcode_name_NXDOMAIN', 
       'traffic_direction_IPv6',
       'traffic_direction_external','traffic_direction_incoming', 
        'traffic_direction_internal','traffic_direction_outgoing']


## http

In [338]:
current_dir_path = '/usr/local/logs/2024-02-12'
if not os.path.islink(current_dir_path):
    # sub_dir is now any given historical data directory 
    logging.info(f"Checking {current_dir_path}")
    for file in os.listdir(current_dir_path):
        # file is now any given file in the historical data directory
        current_file_path = os.path.join(current_dir_path, file)
        if "http." in file: #conn.
            # get the whole file in memory
            logging.info(f"Opening file {current_file_path}")
            json_data_file = ungzip(current_file_path)
            # print(current_file_path)
            # print(json_data_file)

In [339]:
import json 
for line in json_data_file.splitlines():
        # log_entry is now a single json log from the file
        log_entry = json.loads(line.strip())
        print(log_entry)

{'ts': 1707721209.155077, 'uid': 'CGpf9R1APvMKSdDIn2', 'id.orig_h': '192.168.0.109', 'id.orig_p': 57562, 'id.resp_h': '192.168.0.168', 'id.resp_p': 7000, 'trans_depth': 1, 'method': 'GET', 'uri': '/info?txtAirPlay&txtRAOP RTSP/1.', 'version': '0.9', 'request_body_len': 0, 'response_body_len': 1754, 'status_code': 0, 'status_msg': '<empty>', 'tags': [], 'resp_fuids': ['FuhWEF48bHYDzu07H7'], 'resp_mime_types': ['text/plain']}
{'ts': 1707721241.169723, 'uid': 'Cyl3PW2wswIPW1tjY7', 'id.orig_h': '192.168.0.168', 'id.orig_p': 62770, 'id.resp_h': '17.253.3.220', 'id.resp_p': 80, 'trans_depth': 1, 'method': 'CONNECT', 'host': 'proxy-safebrowsing.googleapis.com', 'uri': 'proxy-safebrowsing.googleapis.com:443', 'version': '1.1', 'request_body_len': 0, 'response_body_len': 0, 'status_code': 200, 'status_msg': 'OK', 'tags': [], 'proxied': ['PROXY-CONNECTION -> keep-alive']}
{'ts': 1707723317.563444, 'uid': 'CNKryg21vxMXbO22ud', 'id.orig_h': '192.168.0.168', 'id.orig_p': 62836, 'id.resp_h': '17.253

In [340]:
#utils 
import ipaddress
def one_hot_encode(df, column_name):
    for col in column_name:
        if col in df.columns:
            df = pd.get_dummies(data=df, columns=[col])
    return df

def create_broadcast_variable(new_df):
    # create broadcast variable
    # can have more than one broadcast address
    #255 is the broadcast address for ipv4 
    if 'id.resp_h' in new_df.columns:
        new_df['is_destination_broadcast'] = new_df['id.resp_h'].apply(lambda x: 1 if "255" in x[-3:] else 0) 
    return new_df

def create_direction_variable(new_df):
    # #create traffic direction variable
    new_df['traffic_direction']        = new_df.apply(lambda x: get_traffic_direction(x['id.orig_h'], x['id.resp_h']), axis=1) 
    return new_df


def get_traffic_direction(source_ip, destination_ip):
    """
    Takes a source and destination IP address and returns the direction of the traffic.
    Please ensure the source and destination are correct as this is useless without the verification of the parameters.

    Parameters
    ----------
    source_ip: str
        Source IP address of the flow.
    destination_ip: str
        Destination IP address of the flow.
    
    Returns
    -------
    str: string indicating the direction. Can be 'internal', 'outgoing', 'incoming' or 'external'.
    """
    src_ip = ipaddress.ip_address(source_ip) 
    dest_ip = ipaddress.ip_address(destination_ip) 
    if src_ip.version == 6 or dest_ip.version ==6:
        return "IPv6"
    
    if is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "internal"
    elif is_private_ip(source_ip) and not is_private_ip(destination_ip):
        return "outgoing"
    elif not is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "incoming"
    else:
        return "external"
    

def is_private_ip(ip_str):
    """
    Takes an IP string and returns whether the IP is private or not per RFC 1918.

    Parameters
    ----------
    ip_str: str
        String of an IP address.

    Returns
    -------
    bool: a bool of whether or not the IP is private.
    """
    try:
        ip = ipaddress.ip_address(ip_str)
        if ip.version == 4:
            return ip.is_private
        else:
            return False  # Ignore IPv6 addresses
    except ValueError:
        return False  # Invalid IP address format

def makedf_samecol(cols, new_df):
    #Create these columns if they are not present in the original df and fill them with 0s. 
    # Ensure that all the specified columns are present even if they are not present in the original df. 

    for col in cols:
        if col not in new_df.columns:
            new_df[col] = 0
    return new_df[cols]

In [321]:
features = ['id.orig_h', 'id.resp_h','trans_depth','method','host','version','request_body_len','response_body_len','status_code']
        
data_list = []
for line in json_data_file.splitlines():
    # log_entry is now a single json log from the file
    log_entry = json.loads(line.strip())
    
    # Check if each feature is present in the log_entry
    feature_values = [log_entry.get(feature, None) for feature in features]
    data_list.append(feature_values)

df = pd.DataFrame(data_list, columns=features)

In [341]:
df.head()

id.orig_h        id.resp_h  trans_depth  \
0  192.168.0.109    192.168.0.168            1   
1  192.168.0.168     17.253.3.220            1   
2  192.168.0.168     17.253.3.217            1   
3  192.168.0.168  192.229.211.108            1   
4  192.168.0.168  192.229.211.108            1   

                                host version  request_body_len  \
0                               None     0.9                 0   
1  proxy-safebrowsing.googleapis.com     1.1                 0   
2  proxy-safebrowsing.googleapis.com     1.1                 0   
3                  ocsp.digicert.com     1.1                 0   
4                  ocsp.digicert.com     1.1                 0   

   response_body_len  has_host  is_destination_broadcast  method_CONNECT  \
0               1754         0                         0               0   
1                  0         1                         0               1   
2                  0         1                         0               1   
3                471         1                         0               0   
4                313         1                         0               0   

   method_GET  status_code_0  status_code_200  traffic_direction_internal  \
0           1              1                0                           1   
1           0              0                1                           0   
2           0              0                1                           0   
3           1              0                1                           0   
4           1              0                1                           0   

   traffic_direction_outgoing  
0                           0  
1                           1  
2                           1  
3                           1  
4                           1

In [342]:
#This code checks for null values in each feature
has_null = []
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")    
    if null_count:
        has_null.append(feature)

print("\nhas_null",has_null) # has_null ['host'] 

# Create a variable for each feature that contains null, with the column name "has_null_featurename"
for feature in has_null: 
    df[f'has_{feature}'] = df[feature].notnull().astype(int)


Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for trans_depth: 0
Null count for host: 1
Null count for version: 0
Null count for request_body_len: 0
Null count for response_body_len: 0
Null count for has_host: 0
Null count for is_destination_broadcast: 0
Null count for method_CONNECT: 0
Null count for method_GET: 0
Null count for status_code_0: 0
Null count for status_code_200: 0
Null count for traffic_direction_internal: 0
Null count for traffic_direction_outgoing: 0

has_null ['host']


In [343]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'trans_depth', 'host', 'version',
       'request_body_len', 'response_body_len', 'has_host',
       'is_destination_broadcast', 'method_CONNECT', 'method_GET',
       'status_code_0', 'status_code_200', 'traffic_direction_internal',
       'traffic_direction_outgoing'],
      dtype='object')

In [ ]:
#identify null
#create has null
# create broadcast, traffic_direction variables
# one hot encode categorical variables: 'method','status_code','traffic_direction'
#fillna with 0s: len?? (no na in len)
#same columns  (no 'host)

In [344]:
df = create_broadcast_variable(df)
df = create_direction_variable(df)

In [345]:
column_name = ['version','method','status_code','traffic_direction']
df = one_hot_encode(df, column_name)

In [346]:
df.head()

id.orig_h        id.resp_h  trans_depth  \
0  192.168.0.109    192.168.0.168            1   
1  192.168.0.168     17.253.3.220            1   
2  192.168.0.168     17.253.3.217            1   
3  192.168.0.168  192.229.211.108            1   
4  192.168.0.168  192.229.211.108            1   

                                host  request_body_len  response_body_len  \
0                               None                 0               1754   
1  proxy-safebrowsing.googleapis.com                 0                  0   
2  proxy-safebrowsing.googleapis.com                 0                  0   
3                  ocsp.digicert.com                 0                471   
4                  ocsp.digicert.com                 0                313   

   has_host  is_destination_broadcast  method_CONNECT  method_GET  \
0         0                         0               0           1   
1         1                         0               1           0   
2         1                         0               1           0   
3         1                         0               0           1   
4         1                         0               0           1   

   status_code_0  status_code_200  traffic_direction_internal  \
0              1                0                           1   
1              0                1                           0   
2              0                1                           0   
3              0                1                           0   
4              0                1                           0   

   traffic_direction_outgoing  version_0.9  version_1.1  \
0                           0            1            0   
1                           1            0            1   
2                           1            0            1   
3                           1            0            1   
4                           1            0            1   

   traffic_direction_internal  traffic_direction_outgoing  
0                           1                           0  
1                           0                           1  
2                           0                           1  
3                           0                           1  
4                           0                           1

In [347]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'trans_depth', 'host', 'request_body_len',
       'response_body_len', 'has_host', 'is_destination_broadcast',
       'method_CONNECT', 'method_GET', 'status_code_0', 'status_code_200',
       'traffic_direction_internal', 'traffic_direction_outgoing',
       'version_0.9', 'version_1.1', 'traffic_direction_internal',
       'traffic_direction_outgoing'],
      dtype='object')

In [348]:
df['has_host']

0    0
1    1
2    1
3    1
4    1
5    1
6    1
7    1
Name: has_host, dtype: int64

In [ ]:
#TODO: to be confirmed once EDA is done
http_cols = ['trans_depth', 'request_body_len',
       'response_body_len', 'has_host', 'is_destination_broadcast',
       'method_CONNECT', 'method_GET', 
       'status_code_0', 'status_code_200',
       'version_0.9', 'version_1.1',
       'traffic_direction_IPv6',
       'traffic_direction_internal', 'traffic_direction_outgoing',
       'traffic_direction_internal','traffic_direction_outgoing']
